In [176]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

In [177]:
import numpy as np

In [145]:
ratings = pd.read_excel("hw4.xlsx",index_col=0)

In [146]:
ratings = ratings.iloc[:, 4:54]
ratings
# here serial number is the user id

,The Social Network,A Prophet,Amour,The King's Speech,La La Land,Boyhood,Inception,A Separation,The Artist,The White Ribbon,...,Call Me by Your Name,Winter's Bone,The Grand Budapest Hotel,Dunkirk,Inside Llewyn Davis,A Serious Man,Toy Story 3,Beasts of the Southern Wild,The Imitation Game,The Fighter
Serial Number,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN
3,4.0,NaN,NaN,3.0,4.0,4.0,4.0,NaN,NaN,NaN,...,4.0,NaN,3.0,3.0,4.0,NaN,NaN,NaN,3.0,NaN
4,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,4.0,3.0
5,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,5.0,NaN,2.0,5.0,5.0,NaN,3.0,NaN,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,NaN,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
95,3.0,NaN,NaN,4.0,3.0,4.0,4.0,4.0,3.0,NaN,...,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,5.0,4.0
96,5.0,NaN,NaN,3.0,3.0,3.0,4.0,NaN,4.0,NaN,...,NaN,NaN,4.0,3.0,NaN,NaN,3.0,NaN,5.0,NaN


## Get rid of the null value  
(Aim: To standardize the rating of a user in case a user is too harsh or too lenient

Let's remove movies which have less than 10 users who rated it.
(Aim: reduce the noice of data)
Fill remaining NaN with 0.

In [147]:
ratings = ratings.dropna(thresh=10,axis = 1).fillna(0)

In [190]:
ratings.head()
ratings.columns

Index([' The Social Network', ' The King's Speech', ' La La Land', ' Boyhood',
       ' Inception', ' A Separation', ' The Artist', ' Zero Dark Thirty',
       ' Avatar', ' Spotlight', ' 12 Years a Slave',
       ' Blue is the Warmest Colour', ' Up in the Air',
       ' Inglourious Basterds', ' Mad Max: Fury Road', ' Moonlight',
       ' Birdman', ' Manchester by the Sea', ' Lincoln', ' Hugo',
       ' The Shape of Water', ' Three Billboards Outside Ebbing, Missouri',
       ' Argo', ' Gravity', ' Black Swan', ' The Wolf of Wall Street',
       ' True Grit', ' Life of Pi', ' Arrival', ' Call Me by Your Name',
       ' The Grand Budapest Hotel', ' Dunkirk', ' Toy Story 3',
       ' The Imitation Game', ' The Fighter'],
      dtype='object')

## Standardize the ratings

In [149]:
def standardize(row):
    new_row = (row - row.mean())/(row.max() - row.min())
    return new_row

ratings_std = ratings.apply(standardize)
ratings_std

,The Social Network,The King's Speech,La La Land,Boyhood,Inception,A Separation,The Artist,Zero Dark Thirty,Avatar,Spotlight,...,The Wolf of Wall Street,True Grit,Life of Pi,Arrival,Call Me by Your Name,The Grand Budapest Hotel,Dunkirk,Toy Story 3,The Imitation Game,The Fighter
Serial Number,,,,,,,,,,,,,,,,,,,,,
1,-0.363265,-0.463265,-0.561224,-0.138776,0.277551,-0.091837,-0.093878,-0.161224,0.297959,-0.226531,...,0.430612,-0.1,0.453061,-0.210204,-0.228571,0.563265,-0.395918,-0.426531,0.510204,-0.118367
2,-0.363265,-0.463265,-0.561224,-0.138776,0.277551,-0.091837,-0.093878,-0.161224,-0.702041,-0.226531,...,0.430612,-0.1,-0.546939,-0.210204,-0.228571,-0.436735,-0.395918,0.373469,0.510204,-0.118367
3,0.436735,0.136735,0.238776,0.661224,0.077551,-0.091837,-0.093878,-0.161224,-0.102041,0.373469,...,-0.569388,0.5,0.453061,0.389796,0.571429,0.163265,0.204082,-0.426531,0.110204,-0.118367
4,-0.363265,0.336735,0.438776,-0.138776,-0.722449,-0.091837,0.706122,-0.161224,0.297959,-0.226531,...,0.430612,-0.1,0.053061,-0.210204,-0.228571,-0.436735,0.404082,0.373469,0.310204,0.481633
5,-0.363265,-0.463265,-0.561224,-0.138776,0.277551,-0.091837,-0.093878,-0.161224,-0.502041,-0.226531,...,0.030612,0.9,-0.146939,0.189796,0.771429,-0.036735,0.604082,0.173469,0.310204,0.481633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,-0.363265,0.336735,-0.561224,-0.138776,0.277551,-0.091837,-0.093878,-0.161224,0.297959,-0.226531,...,-0.569388,-0.1,-0.546939,-0.210204,-0.228571,-0.436735,-0.395918,-0.426531,0.510204,-0.118367
95,0.236735,0.336735,0.038776,0.661224,0.077551,0.708163,0.506122,0.638776,-0.102041,0.773469,...,0.230612,0.5,-0.146939,0.789796,-0.228571,0.563265,0.404082,-0.426531,0.510204,0.681633
96,0.636735,0.136735,0.038776,0.461224,0.077551,-0.091837,0.706122,0.238776,-0.102041,-0.226531,...,0.430612,0.5,-0.146939,-0.210204,-0.228571,0.363265,0.204082,0.173469,0.510204,-0.118367


## Cosine similarity 

1. user-to-user similarity

In [150]:
item_similarity_us = cosine_similarity(ratings_std)
item_similarity_us

array([[ 1.        ,  0.18948459, -0.19654328, ..., -0.20481352,
        -0.00828089, -0.08975998],
       [ 0.18948459,  1.        , -0.43623085, ..., -0.14582588,
        -0.22016714, -0.13962809],
       [-0.19654328, -0.43623085,  1.        , ...,  0.25696007,
         0.13483938,  0.4590696 ],
       ...,
       [-0.20481352, -0.14582588,  0.25696007, ...,  1.        ,
        -0.03366765,  0.1161468 ],
       [-0.00828089, -0.22016714,  0.13483938, ..., -0.03366765,
         1.        ,  0.24215203],
       [-0.08975998, -0.13962809,  0.4590696 , ...,  0.1161468 ,
         0.24215203,  1.        ]])

In [182]:
item_similarity_us_0 = cosine_similarity(ratings)
item_similarity_us_0

array([[1.        , 0.49239912, 0.47245559, ..., 0.47961646, 0.57112617,
        0.5568901 ],
       [0.49239912, 1.        , 0.24814583, ..., 0.43034096, 0.41907596,
        0.47007805],
       [0.47245559, 0.24814583, 1.        , ..., 0.68885633, 0.61966068,
        0.78180021],
       ...,
       [0.47961646, 0.43034096, 0.68885633, ..., 1.        , 0.56127055,
        0.65819509],
       [0.57112617, 0.41907596, 0.61966068, ..., 0.56127055, 1.        ,
        0.68956356],
       [0.5568901 , 0.47007805, 0.78180021, ..., 0.65819509, 0.68956356,
        1.        ]])

In [151]:
item_similarity_us_df = pd.DataFrame(item_similarity_us)
item_similarity_us_df

,0,1,2,3,4,5,6,7,8,9,...,88,89,90,91,92,93,94,95,96,97
0,1.000000,0.189485,-0.196543,-0.039224,-0.107647,0.280648,0.161204,0.460185,-0.069896,-0.235782,...,0.137363,-0.149068,0.153007,-0.213386,-0.227348,0.223448,-0.192598,-0.204814,-0.008281,-0.089760
1,0.189485,1.000000,-0.436231,-0.234877,0.362484,0.539489,-0.251890,0.342582,-0.118360,0.161788,...,-0.067900,-0.247536,0.142499,-0.130512,0.100250,0.427663,-0.320059,-0.145826,-0.220167,-0.139628
2,-0.196543,-0.436231,1.000000,-0.159587,0.046508,-0.410044,-0.027731,-0.356373,0.064238,-0.186103,...,0.205986,0.223241,-0.066841,0.387382,-0.063721,-0.380453,0.560105,0.256960,0.134839,0.459070
3,-0.039224,-0.234877,-0.159587,1.000000,-0.163101,-0.270710,0.228424,-0.055538,-0.172771,-0.357665,...,0.208290,-0.199821,-0.022772,-0.034041,-0.149076,-0.260011,0.218915,0.231002,0.079692,-0.050049
4,-0.107647,0.362484,0.046508,-0.163101,1.000000,0.077971,-0.453827,-0.178028,-0.220937,0.116915,...,0.325383,0.045020,-0.014029,0.028396,0.076001,-0.015750,0.101297,0.018782,0.238350,0.014208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.223448,0.427663,-0.380453,-0.260011,-0.015750,0.869846,0.179253,0.231573,0.120954,0.248912,...,-0.303158,-0.111476,0.450164,-0.401000,0.110760,1.000000,-0.423042,-0.256859,-0.440151,-0.346308
94,-0.192598,-0.320059,0.560105,0.218915,0.101297,-0.476501,-0.066507,-0.460834,-0.015057,-0.530602,...,0.488677,-0.120733,-0.167876,0.596210,-0.096620,-0.423042,1.000000,0.451394,0.243606,0.286367
95,-0.204814,-0.145826,0.256960,0.231002,0.018782,-0.288220,-0.004817,-0.141786,0.110970,-0.329364,...,0.207462,-0.082436,-0.260102,0.334524,-0.000592,-0.256859,0.451394,1.000000,-0.033668,0.116147
96,-0.008281,-0.220167,0.134839,0.079692,0.238350,-0.395565,-0.036846,-0.149231,0.062333,-0.046033,...,0.365972,-0.155340,-0.207504,0.435793,0.221998,-0.440151,0.243606,-0.033668,1.000000,0.242152


In [152]:
user_n = np.arange(1,99)
item_similarity_us_df = item_similarity_us_df.set_index(user_n)

In [153]:
item_similarity_us_df.columns = user_n
item_similarity_us_df
export_us = item_similarity_us_df.to_csv(r'D:\DataAnaly\python\bax401-HW4\user_cosine_similarity.csv') 

2. movie-to-movie similarity

In [154]:
item_similarity_m = cosine_similarity(ratings_std.T) # compare target is row, thus transpose df to put movies in row
item_similarity_m

array([[ 1.        ,  0.20676647,  0.19347647, ...,  0.16412921,
         0.37492195,  0.09875429],
       [ 0.20676647,  1.        ,  0.23649456, ..., -0.0134765 ,
         0.1570657 ,  0.15405471],
       [ 0.19347647,  0.23649456,  1.        , ...,  0.12614803,
         0.00573563,  0.00430232],
       ...,
       [ 0.16412921, -0.0134765 ,  0.12614803, ...,  1.        ,
         0.05188695,  0.00446124],
       [ 0.37492195,  0.1570657 ,  0.00573563, ...,  0.05188695,
         1.        ,  0.23306382],
       [ 0.09875429,  0.15405471,  0.00430232, ...,  0.00446124,
         0.23306382,  1.        ]])

In [155]:
item_similarity_m_df = pd.DataFrame(item_similarity_m)
item_similarity_m_df

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1.000000,0.206766,0.193476,0.104360,0.058143,0.108652,0.205710,0.351222,0.134504,0.341477,...,0.313396,0.171279,0.125306,0.034118,0.094871,0.244951,0.200067,0.164129,0.374922,0.098754
1,0.206766,1.000000,0.236495,0.082841,0.083707,0.256689,0.237521,0.084807,0.020255,0.151466,...,0.169312,0.059496,0.230879,0.091785,0.167971,0.387060,0.245451,-0.013476,0.157066,0.154055
2,0.193476,0.236495,1.000000,0.000571,-0.046932,0.095694,0.120354,-0.071548,0.179930,0.001319,...,0.005065,-0.075167,0.253416,0.152282,0.311563,0.184435,0.222024,0.126148,0.005736,0.004302
3,0.104360,0.082841,0.000571,1.000000,0.051181,0.217102,0.124291,0.130663,-0.045943,0.115553,...,0.001288,0.115559,0.141744,0.189681,0.004325,0.162713,0.180498,0.009154,0.101829,0.061961
4,0.058143,0.083707,-0.046932,0.051181,1.000000,0.157233,0.101211,0.176834,0.227319,0.133170,...,0.140580,0.090862,0.297283,0.216497,-0.170268,0.009419,0.180994,-0.015049,0.250275,0.098088
5,0.108652,0.256689,0.095694,0.217102,0.157233,1.000000,0.124768,0.069385,-0.042123,0.280807,...,0.120447,0.063134,0.180544,0.162929,0.014378,0.264414,0.103285,0.033202,0.027562,0.352380
6,0.205710,0.237521,0.120354,0.124291,0.101211,0.124768,1.000000,0.296030,0.085630,0.287129,...,0.152359,0.366393,0.141412,0.181151,-0.014132,0.066213,0.276118,0.086355,0.126295,0.401478
7,0.351222,0.084807,-0.071548,0.130663,0.176834,0.069385,0.296030,1.000000,0.025559,0.457557,...,0.325431,0.275885,0.021284,0.361346,-0.118634,0.163674,0.284251,0.135911,0.343180,0.311896
8,0.134504,0.020255,0.179930,-0.045943,0.227319,-0.042123,0.085630,0.025559,1.000000,-0.053279,...,-0.071455,-0.024360,0.171699,-0.033442,-0.153786,-0.003305,0.024042,0.039822,0.070330,0.010519
9,0.341477,0.151466,0.001319,0.115553,0.133170,0.280807,0.287129,0.457557,-0.053279,1.000000,...,0.334520,0.075785,0.142275,0.298344,-0.013323,0.257331,0.227515,0.055797,0.159366,0.276581


In [156]:
movie_name = ratings_std.columns
item_similarity_m_df = item_similarity_m_df.set_index(movie_name)
item_similarity_m_df.columns = movie_name
item_similarity_m_df
export_m = item_similarity_m_df.to_csv(r'D:\DataAnaly\python\bax401-HW4\movie_cosine_similarity.csv') 

## Make recommendations

In [157]:
def get_similar_movies(movie_name,rating):
    similar_m_score = item_similarity_m_df[movie_name]*(rating-2.5)  
    # for ratings<3, we need the under-preferred result in the bottom of list, thus subtract mean score from 5
    similar_m_score = similar_m_score.sort_values(ascending= False)
    
    return similar_m_score

In [158]:
def get_similar_user(userid):
    similar_user_score = item_similarity_us_df[userid]
    # for ratings<3, we need the under-preferred result in the bottom of list, thus subtract mean score from 5
    similar_user_score = similar_user_score.sort_values(ascending= False)
    
    return similar_user_score 

 ##  
 ## For HaoHao(15)

1) user based selection

In [159]:
print(get_similar_user(15))

15    1.000000
39    0.640864
90    0.386724
47    0.385393
60    0.380085
        ...   
33   -0.397838
31   -0.409195
19   -0.426071
95   -0.433638
52   -0.553203
Name: 15, Length: 98, dtype: float64


the most similar user is 39-HU WENQING, but there's no movie that watched by 39 but not watched by 15.  
Then we turn to movie similarity

2) movie_based selection

In [160]:
member = [( " The King's Speech", 5),(" La La Land",5),(" Avatar",5),(" Life of Pi",5),(" Call Me by Your Name",5),
          (" Manchester by the Sea",2),(" The Shape of Water",4), (" The Grand Budapest Hotel",4),( " Toy Story 3",4)]
similar_movies = pd.DataFrame()
for movie,rating in member:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating))
    
similar_movies.head()
similar_movies.sum().sort_values(ascending=False)

 La La Land                                   5.550898
 The King's Speech                            4.832508
 Life of Pi                                   4.765941
 Call Me by Your Name                         4.114019
 The Grand Budapest Hotel                     3.770143
 The Shape of Water                           3.173887
 Avatar                                       3.160360
 Dunkirk                                      3.126367
 The Social Network                           2.606776
 Hugo                                         2.482560
 Moonlight                                    2.221281
 Black Swan                                   2.178595
 Blue is the Warmest Colour                   2.105254
 Manchester by the Sea                        2.104156
 The Artist                                   2.031202
 Mad Max: Fury Road                           2.007335
 Birdman                                      1.899127
 Three Billboards Outside Ebbing, Missouri    1.898077
 Toy Story

**Thus, recommendation for HAOHAO:Dunkirk, The Social Network, Hugo**

##  
## For Minke(16)

1) user based selection

In [161]:
print(get_similar_user(16))

16    1.000000
77    0.423547
96    0.415036
13    0.371199
9     0.350704
        ...   
54   -0.430012
66   -0.489415
68   -0.490662
38   -0.520477
35   -0.521306
Name: 16, Length: 98, dtype: float64


Similar user: 77-Zhang Shuyu; 96- Alayna Myrick

In [162]:
ratings.loc[77].sort_values(ascending= False)

 Mad Max: Fury Road                           5.0
 The King's Speech                            5.0
 Dunkirk                                      5.0
 The Grand Budapest Hotel                     5.0
 La La Land                                   5.0
 Life of Pi                                   5.0
 Inception                                    5.0
 Birdman                                      5.0
 Black Swan                                   5.0
 12 Years a Slave                             5.0
 Spotlight                                    4.0
 Up in the Air                                4.0
 Hugo                                         4.0
 The Social Network                           4.0
 Argo                                         4.0
 Avatar                                       4.0
 Call Me by Your Name                         4.0
 The Wolf of Wall Street                      4.0
 Manchester by the Sea                        3.0
 Moonlight                                    0.0


In [163]:
ratings.loc[96].sort_values(ascending= False)

 The Social Network                           5.0
 The Wolf of Wall Street                      5.0
 12 Years a Slave                             5.0
 The Imitation Game                           5.0
 Lincoln                                      4.0
 The Grand Budapest Hotel                     4.0
 Inception                                    4.0
 The Artist                                   4.0
 Blue is the Warmest Colour                   4.0
 The Shape of Water                           4.0
 The King's Speech                            3.0
 La La Land                                   3.0
 Boyhood                                      3.0
 Avatar                                       3.0
 Birdman                                      3.0
 Manchester by the Sea                        3.0
 Argo                                         3.0
 Toy Story 3                                  3.0
 Dunkirk                                      3.0
 Black Swan                                   3.0


2) movie_based selection

In [164]:
member = [(" The Social Network",5),( " The King's Speech", 5),(" La La Land",5),(" Boyhood",5),(" Inception",5),
          (" A Separation",4),(" Avatar",5),(" Life of Pi",5),(" The Grand Budapest Hotel",5),
          (" Mad Max: Fury Road",4),(" Birdman",4),(" Manchester by the Sea",5),(" Hugo",5), 
          (" Black Swan",5),(" The Wolf of Wall Street",5),(" The Imitation Game",5),( " Toy Story 3",5),
         (" 12 Years a Slave",4), (" A Separation", 4)]
for movie,rating in member:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating))
    
similar_movies.head()
similar_movies.sum().sort_values(ascending=False)

 The King's Speech                            14.252650
 The Grand Budapest Hotel                     13.885643
 Life of Pi                                   13.002029
 La La Land                                   12.744444
 Hugo                                         12.157679
 The Social Network                           12.150232
 Dunkirk                                      12.007450
 Birdman                                      11.272713
 Manchester by the Sea                        10.895811
 A Separation                                 10.622398
 Mad Max: Fury Road                           10.329080
 Black Swan                                   10.248514
 12 Years a Slave                             10.042813
 Lincoln                                       9.535491
 Blue is the Warmest Colour                    9.284970
 The Imitation Game                            9.221042
 The Fighter                                   9.185720
 The Wolf of Wall Street                       9

**Recommendation: Dunkirk; Mad Max, Fury Road; Lincoln**

##   
## FOR WHN

1) user based selection

In [165]:
get_similar_user(35).head(10)

35    1.000000
38    0.921301
66    0.883388
68    0.852630
6     0.845704
54    0.799318
36    0.696702
26    0.687951
79    0.684149
94    0.680948
Name: 35, dtype: float64

In [166]:
ratings.loc[38].sort_values(ascending= False)

 Avatar                                       3.0
 The Fighter                                  0.0
 Mad Max: Fury Road                           0.0
 Inglourious Basterds                         0.0
 Up in the Air                                0.0
 Blue is the Warmest Colour                   0.0
 12 Years a Slave                             0.0
 Spotlight                                    0.0
 Zero Dark Thirty                             0.0
 Birdman                                      0.0
 The Artist                                   0.0
 A Separation                                 0.0
 Inception                                    0.0
 Boyhood                                      0.0
 La La Land                                   0.0
 The King's Speech                            0.0
 Moonlight                                    0.0
 Manchester by the Sea                        0.0
 The Imitation Game                           0.0
 True Grit                                    0.0


In [167]:
ratings.loc[66].sort_values(ascending= False)

 The Social Network                           1.0
 The Wolf of Wall Street                      1.0
 Mad Max: Fury Road                           1.0
 Moonlight                                    1.0
 Spotlight                                    1.0
 Avatar                                       1.0
 The Shape of Water                           1.0
 Black Swan                                   1.0
 Inglourious Basterds                         1.0
 Inception                                    1.0
 Life of Pi                                   1.0
 Arrival                                      1.0
 Boyhood                                      1.0
 La La Land                                   1.0
 Toy Story 3                                  1.0
 Up in the Air                                0.0
 The Artist                                   0.0
 Zero Dark Thirty                             0.0
 12 Years a Slave                             0.0
 Blue is the Warmest Colour                   0.0


In [168]:
ratings.loc[68].sort_values(ascending= False)

 Toy Story 3                                  2.0
 The Grand Budapest Hotel                     2.0
 The Imitation Game                           2.0
 Black Swan                                   1.0
 Dunkirk                                      1.0
 Call Me by Your Name                         1.0
 Avatar                                       1.0
 Gravity                                      1.0
 The Shape of Water                           1.0
 The Artist                                   0.0
 12 Years a Slave                             0.0
 Spotlight                                    0.0
 Zero Dark Thirty                             0.0
 The Fighter                                  0.0
 A Separation                                 0.0
 Up in the Air                                0.0
 Inception                                    0.0
 Boyhood                                      0.0
 La La Land                                   0.0
 The King's Speech                            0.0


*whn has rated for very few movies, and many of them are below 3. She's a quite harsh rater!*

2) movie_based selection

In [169]:
member = [(" Inception",3),(" Avatar",1),(" The Imitation Game",1)]
for movie,rating in member:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating))
    
similar_movies.head()
similar_movies.sum().sort_values(ascending=False)

 The King's Speech                            14.028522
 The Grand Budapest Hotel                     13.574349
 Life of Pi                                   12.704939
 La La Land                                   12.442480
 Hugo                                         11.667999
 Dunkirk                                      11.588877
 The Social Network                           11.415164
 Birdman                                      11.010289
 Manchester by the Sea                        10.831857
 A Separation                                 10.722857
 Black Swan                                   10.079894
 Mad Max: Fury Road                            9.885745
 12 Years a Slave                              9.663722
 Lincoln                                       9.196025
 Blue is the Warmest Colour                    9.104074
 The Wolf of Wall Street                       8.921018
 The Fighter                                   8.869390
 Moonlight                                     8

**Recommendation for WHN is quite difficult since we don't know what kind of movie does she like, and few of the other people has similar rating behavior with her. After I talk to WHN, she realized that she has given opposite rating where she thought 1 is the highest rating.**

##  
## For Stella

1) user based selection

In [170]:
get_similar_user(48).head(10)

48    1.000000
60    0.520820
32    0.467342
8     0.450154
67    0.426259
15    0.338264
1     0.299597
39    0.279406
25    0.278292
47    0.272786
Name: 48, dtype: float64

In [174]:
ratings.loc[60].sort_values(ascending= False)

 The Wolf of Wall Street                      5.0
 La La Land                                   5.0
 Call Me by Your Name                         4.0
 Life of Pi                                   4.0
 Zero Dark Thirty                             0.0
 Inglourious Basterds                         0.0
 Up in the Air                                0.0
 Blue is the Warmest Colour                   0.0
 12 Years a Slave                             0.0
 Spotlight                                    0.0
 Avatar                                       0.0
 The Fighter                                  0.0
 The Artist                                   0.0
 Moonlight                                    0.0
 A Separation                                 0.0
 Inception                                    0.0
 Boyhood                                      0.0
 The King's Speech                            0.0
 Mad Max: Fury Road                           0.0
 Manchester by the Sea                        0.0


In [175]:
ratings.loc[32].sort_values(ascending= False)

 Manchester by the Sea                        5.0
 The Wolf of Wall Street                      5.0
 The Social Network                           4.0
 La La Land                                   3.0
 Three Billboards Outside Ebbing, Missouri    3.0
 Avatar                                       2.0
 Dunkirk                                      0.0
 Mad Max: Fury Road                           0.0
 The King's Speech                            0.0
 Boyhood                                      0.0
 Inception                                    0.0
 A Separation                                 0.0
 The Artist                                   0.0
 Zero Dark Thirty                             0.0
 Spotlight                                    0.0
 12 Years a Slave                             0.0
 Blue is the Warmest Colour                   0.0
 Up in the Air                                0.0
 Inglourious Basterds                         0.0
 Moonlight                                    0.0


In [172]:
member = [(" The Social Network",5),(" La La Land",5),(" Avatar",4),(" Blue is the Warmest Colour",4),
          (" Three Billboards Outside Ebbing, Missouri",5),(" The Wolf of Wall Street",5),
          (" The Grand Budapest Hotel",5),(" Life of Pi",5),(" Call Me by Your Name",4)
          ]
for movie,rating in member:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating))
    
similar_movies.head()
similar_movies.sum().sort_values(ascending=False)

 The Grand Budapest Hotel                     18.979443
 Life of Pi                                   18.115535
 The King's Speech                            17.996744
 La La Land                                   17.609569
 The Social Network                           17.537824
 Dunkirk                                      16.341047
 Hugo                                         14.848536
 Manchester by the Sea                        14.596865
 The Wolf of Wall Street                      14.073735
 Birdman                                      14.033914
 Blue is the Warmest Colour                   13.996440
 Three Billboards Outside Ebbing, Missouri    13.633896
 A Separation                                 13.437537
 Mad Max: Fury Road                           13.116424
 Black Swan                                   12.688131
 Lincoln                                      12.602668
 12 Years a Slave                             12.331630
 Moonlight                                    12

**Recommendation for Stella:  The King's Speech; Dunkirk, Hugo**